In [65]:
from datetime import datetime

from google.cloud import aiplatform
import os

In [66]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

In [67]:
PROJECT_ID

'qwiklabs-asl-00-6fcd414e1f60'

In [77]:
# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


In [73]:
!pip install jsonschema



In [75]:
!pip install nodeenv
!pip install pre-commit

In [169]:
# !cat trainer_image_vertex/Dockerfile

In [170]:
# !cat trainer_image_vertex/model.py

In [171]:
%%bash
ls /home/jupyter/.local/bin

dsl-compile
dsl-compile-v2
kfp


In [172]:
import os

In [173]:
%%bash
pwd

#cp /home/jupyter/asl-ml-immersion/notebooks/capstone_project/CapStone_Phase_Contrast/src
 

/home/jupyter/CapStone_Phase_Contrast


In [175]:
os.chdir('trainer_image_vertex')
os.getcwd()

'/home/jupyter/CapStone_Phase_Contrast/trainer_image_vertex'

## name docker image

In [176]:
IMAGE_NAME = "Dockerfile"
TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}"
TRAINING_CONTAINER_IMAGE_URI


'gcr.io/qwiklabs-asl-00-6fcd414e1f60/docker_image_kamrul:latest'

In [178]:

SERVING_CONTAINER_IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-8:latest"
SERVING_CONTAINER_IMAGE_URI

'us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-8:latest'

In [179]:
%%bash
ls

In [180]:

pwd

'/home/jupyter/CapStone_Phase_Contrast/trainer_image_vertex'

In [181]:
os.chdir("/home/jupyter/CapStone_Phase_Contrast/trainer_image_vertex")
os.getcwd()

'/home/jupyter/CapStone_Phase_Contrast/trainer_image_vertex'

## build docker image

In [103]:
!gcloud builds submit --timeout 15m --tag $TRAINING_CONTAINER_IMAGE_URI trainer_image_vertex

Creating temporary tarball archive of 6 file(s) totalling 15.4 KiB before compression.
Uploading tarball of [trainer_image_vertex] to [gs://qwiklabs-asl-00-6fcd414e1f60_cloudbuild/source/1680585442.397761-d0a922fbb990474fa48f360ad67072c2.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-00-6fcd414e1f60/locations/global/builds/cb86c5fb-e430-48ec-b7c5-843f41d0c21a].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/cb86c5fb-e430-48ec-b7c5-843f41d0c21a?project=955332457859 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "cb86c5fb-e430-48ec-b7c5-843f41d0c21a"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-00-6fcd414e1f60_cloudbuild/source/1680585442.397761-d0a922fbb990474fa48f360ad67072c2.tgz#1680585442675192
Copying gs://qwiklabs-asl-00-6fcd414e1f60_cloudbuild/source/1680585442.397761-d0a922fbb990474fa48f360ad67072c2.tgz#1680585442675192...
/ [1 files][  3.2 KiB/  3.2 KiB]         

### setup environment variables for pipeline

In [134]:
ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATA_ROOT = f"{ARTIFACT_STORE}/data"

TRAINING_FILE_PATH = f"{DATA_ROOT}/training/dataset.csv"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/validation/dataset.csv"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BASE_OUTPUT_DIR = f"{ARTIFACT_STORE}/models/{TIMESTAMP}"

%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}
%env BASE_OUTPUT_DIR={BASE_OUTPUT_DIR}
%env MAX_TRIAL_COUNT={1}

env: PIPELINE_ROOT=gs://qwiklabs-asl-00-6fcd414e1f60-kfp-artifact-store/pipeline
env: PROJECT_ID=qwiklabs-asl-00-6fcd414e1f60
env: REGION=us-central1
env: SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-8:latest
env: TRAINING_CONTAINER_IMAGE_URI=gcr.io/qwiklabs-asl-00-6fcd414e1f60/docker_image_kamrul:latest
env: TRAINING_FILE_PATH=gs://qwiklabs-asl-00-6fcd414e1f60-kfp-artifact-store/data/training/dataset.csv
env: VALIDATION_FILE_PATH=gs://qwiklabs-asl-00-6fcd414e1f60-kfp-artifact-store/data/validation/dataset.csv
env: BASE_OUTPUT_DIR=gs://qwiklabs-asl-00-6fcd414e1f60-kfp-artifact-store/models/20230404060703
env: MAX_TRIAL_COUNT=1


Let us make sure that the `ARTIFACT_STORE` has been created, and let us create it if not:

In [135]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-asl-00-6fcd414e1f60-kfp-artifact-store/


In [137]:

!gsutil ls | grep ^{ARTIFACT_STORE}/$


gs://qwiklabs-asl-00-6fcd414e1f60-kfp-artifact-store/


#### Use the CLI compiler to compile the pipeline

In [126]:
# PIPELINE_JSON = "kamrul_kfp_pipeline.json"

In [148]:
!cat pipeline_vertex/pipeline_prebuilt.py

# Copyright 2021 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License"); you may not
# use this file except in compliance with the License. You may obtain a copy of
# the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS"
# BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
# express or implied. See the License for the specific language governing
# permissions and limitations under the License.
"""Kubeflow Covertype Pipeline."""
import os

from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google_cloud_pipeline_components.aiplatform import (
    EndpointCreateOp,
    ModelDeployOp,
    ModelUploadOp,
)
from google_cloud_pipeline_components.experimental import (
    hyperparameter_tuning_job,
)
from google_cloud_pipeline_components.experimental.custom_job import (
    CustomTrainingJobOp,
)
from kfp.v

In [164]:
PIPELINE_JSON = "ct_contrast_kfp_pipeline.json"

In [165]:
!dsl-compile-v2 --py pipeline_vertex/pipeline_prebuilt.py --output $PIPELINE_JSON


/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [166]:
!head {PIPELINE_JSON}

{
  "pipelineSpec": {
    "components": {
      "comp-custom-training-job": {
        "executorLabel": "exec-custom-training-job",
        "inputDefinitions": {
          "parameters": {
            "base_output_directory": {
              "type": "STRING"
            },


In [152]:
# import os
# from kfp.v2 import compiler

# compiler.Compiler().compile(
#     pipeline_func=create_pipeline, 
#     package_path=PIPELINE_JSON,
# )

**Note:** You can also use the Python SDK to compile the pipeline:

```python
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=create_pipeline, 
    package_path=PIPELINE_JSON,
)

```

In [167]:
!head {PIPELINE_JSON}

{
  "pipelineSpec": {
    "components": {
      "comp-custom-training-job": {
        "executorLabel": "exec-custom-training-job",
        "inputDefinitions": {
          "parameters": {
            "base_output_directory": {
              "type": "STRING"
            },


### Deploy the pipeline package

In [168]:
aiplatform.init(project=PROJECT_ID, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name="ct_contrast_kfp_pipeline",
    template_path=PIPELINE_JSON,
    enable_caching=True,
)

pipeline.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/955332457859/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230405160213
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/955332457859/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230405160213')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/ct-phase-contrast-kfp-pipeline-20230405160213?project=955332457859
PipelineJob projects/955332457859/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230405160213 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/955332457859/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230405160213 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/955332457859/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230405160213 current state:
PipelineState.PIPE

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [hyperparameter-tuning-job].; Job (project_id = qwiklabs-asl-00-6fcd414e1f60, job_id = 4479951881036103680) is failed due to the above error.; Failed to handle the job: {project_number = 955332457859, job_id = 4479951881036103680}"
